# Covid Use Case Step 4 - 5 Targeted Attack with nVITA

In [1]:
import argparse
import os
import time
from pathlib import Path

import numpy as np
import nvita.models.train as mt
import torch
from nvita.attacks.bim import BIMTSF
from nvita.attacks.brnv import BRNV
from nvita.attacks.brs import BRS
from nvita.attacks.fgsm import FGSMTSF
from nvita.attacks.fullvita import FULLVITA
from nvita.attacks.nvita import NVITA
from nvita.attacks.utils import (append_result_to_csv_file,
                                 check_result_file_path,
                                 create_empty_result_csv_file)
from nvita.models.data import SplittedTSData
from nvita.models.utils import load_model
from nvita.utils import open_json

D:\Programming\IDE\Anaconda\envs\hons\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()

my_metadata = open_json(os.path.join(
    PATH_ROOT, "experiments", "metadata.json"))
print(my_metadata)

{'data': ['Electricity', 'NZTemp', 'CNYExch', 'Oil'], 'seeds': ['2210', '9999', '58361', '789789', '1111111'], 'y_col_name': ['Consumption', 'Auckland', 'Close', 'Close'], 'window_size': ['4', '3', '7', '7'], 'models': ['CNN', 'LSTM', 'GRU', 'RF'], 'attacks': ['NOATTACK', 'BRS', 'BRNV', 'FGSM', 'BIM', 'NVITA', 'FULLVITA'], 'n_values': [1, 3, 5], 'epsilons': [0.05, 0.1, 0.15, 0.2], 'targets': ['Positive', 'Negative']}


In [3]:
df_name = "CovidUSA"
seed = int(my_metadata["seeds"][0])

In [4]:
s_data = SplittedTSData()
s_data = s_data.load_splitted_data(PATH_ROOT, df_name, seed)
m = load_model(PATH_ROOT, df_name, seed, "LSTM")

In [11]:
ori_Xs, ori_pred_ys, per_X, adv_Xs, adv_pred_ys = [], [], [], [], []

#for test_ind in range(s_data.X_test.shape[0]):
for test_ind in range(60):
    X_current = torch.reshape(s_data.X_test[test_ind], s_data.single_X_shape)
    ground_truth_y = torch.reshape(s_data.y_test[test_ind], s_data.single_y_shape)
    window_range = s_data.window_ranges[test_ind]
    mean_pred, std_pred = mt.get_mean_and_std_pred(m, X_current)
    attack_goal = -0.5
    attack_goal_tup = (test_ind, attack_goal)
    
    n = 5
    epsilon = 0.2
    
    att = NVITA(n, epsilon, m, targeted=True)
    X_adv, _ = att.attack(X_current, attack_goal_tup, window_range, seed=seed, tol=0.0001)
    
    adv_y_pred = mt.adv_predict(m, X_adv)
    eta = X_adv - X_current
    
    ori_Xs.append(X_current)
    ori_pred_ys.append(mean_pred)
    per_X.append(eta)
    adv_Xs.append(X_adv)
    adv_pred_ys.append(adv_y_pred)

In [12]:
ori_pred_ys

[-0.0023476794362068177,
 0.052753646560013294,
 0.054820462502539156,
 0.09583654433488846,
 0.0043006626516580585,
 0.08047312803566456,
 0.02438536275178194,
 0.026922239400446416,
 0.10961608231067657,
 0.040953089594841,
 0.07990269981324673,
 0.5627750891447068,
 0.017750924490392207,
 0.05356514576822519,
 0.06242462396621704,
 0.07853981129825115,
 0.09516053408384323,
 -0.0031928293406963348,
 0.0347275585308671,
 0.02690111391246319,
 0.13520674720406534,
 0.05102775316685438,
 0.09966786704957485,
 0.12955525934696197,
 0.00758990190923214,
 0.04110379841178655,
 0.001815917193889618,
 0.006507706046104431,
 0.06972272329032421,
 0.05011216379702091,
 0.09393323875963688,
 -0.00041560865938663485,
 0.04052451115101576,
 0.0871639173477888,
 0.03458991762250662,
 0.05899327255785465,
 0.12682077832520008,
 0.11604297392070294,
 -0.0008642923086881638,
 0.03544128738343716,
 0.05132087878882885,
 0.04009145226329565,
 0.06937865681946277,
 0.026478569582104684,
 0.031205204650

In [13]:
adv_pred_ys

[-0.0026843127608299254,
 0.041101394370198246,
 0.01040993519127369,
 0.07364232532680035,
 0.004295541644096374,
 0.0761639979481697,
 0.023084690496325493,
 0.024317868761718275,
 0.1056345235556364,
 0.03881384387612343,
 0.06187696561217308,
 0.36157714694738385,
 0.01648517746478319,
 0.045527995824813844,
 0.04878129482269287,
 0.06949200704693795,
 0.08019609123468399,
 -0.00398631788790226,
 0.0251083692535758,
 0.025858742222189902,
 0.10588260352611542,
 0.045412936359643934,
 0.07366038598120213,
 0.10973387777805328,
 0.006831246390938759,
 0.033557885177433494,
 0.0017610155791044236,
 -0.00706537663936615,
 0.04929861083626747,
 0.04333710059523582,
 0.08965672485530377,
 -0.00044606104493141176,
 0.028371800407767295,
 0.07304706700146198,
 0.031574437953531743,
 0.045197544656693935,
 0.1243607945740223,
 0.10779676459729672,
 -0.0011016826331615448,
 0.029491617679595947,
 0.04834665179252624,
 0.03285454113036394,
 0.049461561739444736,
 0.02038001049309969,
 0.00744

In [14]:
np.array(ori_pred_ys) - np.array(adv_pred_ys)

array([3.36633325e-04, 1.16522522e-02, 4.44105273e-02, 2.21942190e-02,
       5.12100756e-06, 4.30913009e-03, 1.30067226e-03, 2.60437064e-03,
       3.98155876e-03, 2.13924572e-03, 1.80257342e-02, 2.01197942e-01,
       1.26574703e-03, 8.03714994e-03, 1.36433291e-02, 9.04780425e-03,
       1.49644428e-02, 7.93488547e-04, 9.61918928e-03, 1.04237169e-03,
       2.93241437e-02, 5.61481681e-03, 2.60074811e-02, 1.98213816e-02,
       7.58655518e-04, 7.54591323e-03, 5.49016148e-05, 1.35730827e-02,
       2.04241125e-02, 6.77506320e-03, 4.27651390e-03, 3.04523855e-05,
       1.21527107e-02, 1.41168503e-02, 3.01547967e-03, 1.37957279e-02,
       2.45998375e-03, 8.24620932e-03, 2.37390324e-04, 5.94966970e-03,
       2.97422700e-03, 7.23691113e-03, 1.99170951e-02, 6.09855909e-03,
       2.37590875e-02, 2.89191537e-03, 3.60152964e-03, 5.01238134e-03,
       9.57199834e-03, 2.68854402e-03, 4.95703891e-03, 1.36849917e-03,
       5.72025873e-03, 1.75731072e-02, 4.13018275e-03, 2.94700753e-02,
      

In [22]:
r = np.array(ori_pred_ys) - np.array(adv_pred_ys)
per_list = []
for i in range(60):
    per = r[i]/s_data.window_ranges[i][0].item()
    per_list.append(per)

In [23]:
sum(per_list)/len(per_list)

0.14381435234492015

In [20]:
s_data.window_ranges

[tensor([0.0185, 0.0821, 0.0183, 0.0069, 0.1220, 0.0801, 0.0006, 0.0000, 0.0000]),
 tensor([5.7581e-02, 6.1747e-01, 6.1838e-02, 3.7893e-02, 4.0251e-01, 1.7283e-01,
         4.0858e-04, 0.0000e+00, 0.0000e+00]),
 tensor([0.1160, 0.3937, 0.0437, 0.0416, 0.4129, 0.4738, 0.1263, 0.0000, 0.0000]),
 tensor([1.0667e-01, 6.2546e-01, 1.3176e-01, 1.3223e-01, 4.0172e-01, 8.1924e-02,
         6.1287e-04, 0.0000e+00, 0.0000e+00]),
 tensor([0.0138, 0.0944, 0.0170, 0.0130, 0.1759, 0.1575, 0.0000, 0.0000, 0.0000]),
 tensor([0.0881, 0.5105, 0.0868, 0.0585, 0.3874, 0.1662, 0.0006, 0.0000, 0.0000]),
 tensor([0.0188, 0.1147, 0.0265, 0.0253, 0.2926, 0.4165, 0.0566, 0.0000, 0.0000]),
 tensor([0.0175, 0.2343, 0.0262, 0.0155, 0.3546, 0.5436, 0.0000, 0.0000, 0.0000]),
 tensor([0.0454, 0.5210, 0.0808, 0.0814, 0.3637, 0.3426, 0.0000, 0.0000, 0.0000]),
 tensor([0.0235, 0.1540, 0.0112, 0.0160, 0.3369, 0.5496, 0.0016, 0.0000, 0.0000]),
 tensor([0.1352, 0.2655, 0.0088, 0.0093, 0.2460, 0.0646, 0.1136, 0.0000, 0.0000]